In [2]:
import cv2
import numpy as np
def drawRec(biggestNew, mainFrame):
        cv2.line(mainFrame, (biggestNew[0][0][0], biggestNew[0][0][1]), (biggestNew[1][0][0], biggestNew[1][0][1]), (0, 255, 0), 20)
        cv2.line(mainFrame, (biggestNew[0][0][0], biggestNew[0][0][1]), (biggestNew[2][0][0], biggestNew[2][0][1]), (0, 255, 0), 20)
        cv2.line(mainFrame, (biggestNew[3][0][0], biggestNew[3][0][1]), (biggestNew[2][0][0], biggestNew[2][0][1]), (0, 255, 0), 20)
        cv2.line(mainFrame, (biggestNew[3][0][0], biggestNew[3][0][1]), (biggestNew[1][0][0], biggestNew[1][0][1]), (0, 255, 0), 20)

#initializing
img = cv2.imread('image.jpg')
img = cv2.resize(img, (int(480*2), int(640*2)))
w, h = 480, 640
imgWarp = img.copy()
# write code here
GrayImg = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
BlurImg = cv2.GaussianBlur(GrayImg, (5, 5), 1)
CannyImg = cv2.Canny(BlurImg, 200, 200)
contours, _ = cv2.findContours(CannyImg, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
ContourImg = img.copy()
ContourImg = cv2.drawContours(ContourImg, contours, -1, (255, 0, 255), 4)
CornerImg = img.copy()
maxArea = 0
biggest = []
for i in contours :
    area = cv2.contourArea(i)
    if area > 500 :
        peri = cv2.arcLength(i, True)
        edges = cv2.approxPolyDP(i, 0.02*peri, True)
        if area > maxArea and len(edges) == 4 :
            biggest = edges
            maxArea = area
if len(biggest) != 0 :
    biggest = biggest.reshape((4, 2))
    biggestNew = np.zeros((4, 1, 2), dtype= np.int32)
    add = biggest.sum(1)
    biggestNew[0] = biggest[np.argmin(add)]
    biggestNew[3] = biggest[np.argmax(add)]
    dif = np.diff(biggest, axis = 1)
    biggestNew[1] = biggest[np.argmin(dif)]
    biggestNew[2] = biggest[np.argmax(dif)]
    drawRec(biggestNew, CornerImg)
    CornerImg = cv2.drawContours(CornerImg, biggestNew, -1, (255, 0, 255), 25)
    pts1 = np.float32(biggestNew)
    pts2 = np.float32([[0, 0], [w, 0], [0, h], [w, h]])
    matrix = cv2.getPerspectiveTransform(pts1, pts2)
    imgWarp = cv2.warpPerspective(img, matrix, (w, h))
#resizing
img = cv2.resize(img,(480, 640))
GrayImg = cv2.resize(GrayImg, (480, 640))
BlurImg = cv2.resize(BlurImg, (480, 640))
CannyImg = cv2.resize(CannyImg, (480, 640))
ContourImg = cv2.resize(ContourImg, (480, 640))
CornerImg = cv2.resize(CornerImg, (480, 640))

#displaying
cv2.imshow("Original Image", img)
cv2.imshow("Gray Scale", GrayImg)
cv2.imshow("Blurred Image", BlurImg)
cv2.imshow("Cannt Edge Detection", CannyImg)
cv2.imshow("Contour Detection", ContourImg)
cv2.imshow("Corner Detection", CornerImg)
cv2.imshow("Output Image", imgWarp)
cv2.waitKey(0)
cv2.destroyAllWindows() 